In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import nx_cugraph as nxcg
from time import time
import os
import cupy as cp
import random

In [2]:
FILENAME = "/home/daniel/Documents/phd/phd-thesis-lab/12-third_year/00-Data/05-actor-movie/actor-movie.graphml"


In [4]:


print(f"using networkx version {nx.__version__}")

G = nx.read_graphml(FILENAME)
print("|N|",G.number_of_nodes(), "   |M|",G.number_of_edges())

nxcg_G = nxcg.from_networkx(G) 

a = time()
#bc_results = nx.betweenness_centrality(G, k=100)
nx.betweenness_centrality(nxcg_G, k=100)
b = time() - a
print("GPU - time: %.10f seconds." % b)

a = time()
bc_results = nx.betweenness_centrality(G, k=100)
#nx.betweenness_centrality(nxcg_G, k=1000)
b = time() - a
print("CPU - time: %.10f seconds." % b)

using networkx version 3.2
|N| 511463    |M| 1470404
GPU - time: 1.2944035530 seconds.


KeyboardInterrupt: 

In [5]:
%%time
sample_size = 100  # Número de nodos de muestreo para aproximar APL
all_nodes = list(G.nodes())
sample_nodes = random.sample(all_nodes, sample_size)

# -------------------------------------------
# 3️⃣ Calcular distancias usando SSSP y acumular
# -------------------------------------------
total_distances = 0
reachable_pairs = 0

for i, src_node in enumerate(sample_nodes):
    # Ejecutar SSSP desde el nodo fuente
    sssp_df = nxcg.all_pairs_shortest_path_length(nxcg_G, source=src_node)
    a = list(sssp_df.values())
    if 0 in a:
      a.remove(0)
    if cp.inf in a:
      a.remove(cp.inf)

    # Acumular suma de distancias y contar cuántos pares fueron alcanzables
    total_distances += sum(a)
    reachable_pairs += len(a)

    #if (i + 1) % 10 == 0:
    #    print(f"Muestreo {i+1}/{sample_size} completado.")

# -------------------------------------------
# 4️⃣ Calcular APL aproximado
# -------------------------------------------
apl_approx = float(total_distances) / reachable_pairs

print(f"\n🔹 APL aproximado con {sample_size} nodos de muestreo: {apl_approx:.4f}")
print(f"🔹 Pares alcanzables contabilizados: {reachable_pairs}")


TypeError: all_pairs_shortest_path_length() got an unexpected keyword argument 'source'

In [177]:
%%time
G = nx.read_graphml(FILENAME)
print("|N|",G.number_of_nodes(), "   |M|",G.number_of_edges())

nxcg_G = nxcg.from_networkx(G) 

sample_size = 100  # Número de nodos de muestreo para aproximar APL
all_nodes = list(G.nodes())
sample_nodes = random.sample(all_nodes, sample_size)

# -------------------------------------------
# 3️⃣ Calcular distancias usando SSSP y acumular
# -------------------------------------------
total_distances = 0
reachable_pairs = 0
c = 0
for i, src_node in enumerate(sample_nodes):
    
    # Ejecutar SSSP desde el nodo fuente
    sssp_df = nx.shortest_path_length(G, source=src_node)


    # Acumular suma de distancias y contar cuántos pares fueron alcanzables
    total_distances += sum(sssp_df.values())
    reachable_pairs += len(sssp_df.values())

# -------------------------------------------
# 4️⃣ Calcular APL aproximado
# -------------------------------------------
apl_approx = float(total_distances) / reachable_pairs

print(f"\n🔹 APL aproximado con {sample_size} nodos de muestreo: {apl_approx:.4f}")
print(f"🔹 Pares alcanzables contabilizados: {reachable_pairs}")


|N| 511463    |M| 1470404

🔹 APL aproximado con 100 nodos de muestreo: 7.0303
🔹 Pares alcanzables contabilizados: 48395737
CPU times: user 1min 4s, sys: 430 ms, total: 1min 5s
Wall time: 1min 5s


In [3]:
%%time
G = nx.read_graphml(FILENAME)
print("|N|",G.number_of_nodes(), "   |M|",G.number_of_edges())

nxcg_G = nxcg.from_networkx(G) 

sample_size = 100  # Número de nodos de muestreo para aproximar APL
all_nodes = list(G.nodes())
sample_nodes = random.sample(all_nodes, sample_size)

# -------------------------------------------
# 3️⃣ Calcular distancias usando SSSP y acumular
# -------------------------------------------
total_distances = 0
reachable_pairs = 0
c = 0
for i, src_node in enumerate(sample_nodes):
    # Ejecutar SSSP desde el nodo fuente

    sssp_df = nxcg.shortest_path_length(nxcg_G, source=src_node)

    # Acumular suma de distancias y contar cuántos pares fueron alcanzables
    total_distances += sum(sssp_df.values())
    reachable_pairs += len(sssp_df.values())

# -------------------------------------------
# 4️⃣ Calcular APL aproximado
# -------------------------------------------
apl_approx = float(total_distances) / reachable_pairs

print(f"\n🔹 APL aproximado con {sample_size} nodos de muestreo: {apl_approx:.4f}")
print(f"🔹 Pares alcanzables contabilizados: {reachable_pairs}")


|N| 511463    |M| 1470404

🔹 APL aproximado con 100 nodos de muestreo: 7.1675
🔹 Pares alcanzables contabilizados: 46400073
CPU times: user 21.9 s, sys: 1.68 s, total: 23.6 s
Wall time: 23.6 s


In [4]:
def convertir_funcion_apl(lista):
    new_list = []
    for i in lista:
        size_ = len(i)
        if size_ > 0:
            new_list.append(len(i)-1)
    return new_list

In [6]:
nxcg.average_clustering(nxcg_G)

0.0

In [5]:
%%time
import igraph as ig
g = ig.read(filename=FILENAME)

print("|N|",g.vcount, "   |M|",g.ecount)


sample_size = 100  # Número de nodos de muestreo para aproximar APL
all_nodes = list(g.vs())
sample_nodes = random.sample(all_nodes, sample_size)

# -------------------------------------------
# 3️⃣ Calcular distancias usando SSSP y acumular
# -------------------------------------------
total_distances = 0
reachable_pairs = 0
c = 0
for i, src_node in enumerate(sample_nodes):
    # Ejecutar SSSP desde el nodo fuente

    sssp_df =convertir_funcion_apl(g.get_shortest_paths(v=src_node))


    # Acumular suma de distancias y contar cuántos pares fueron alcanzables
    total_distances += sum(sssp_df)
    reachable_pairs += len(sssp_df)

# -------------------------------------------
# 4️⃣ Calcular APL aproximado
# -------------------------------------------
apl_approx = float(total_distances) / reachable_pairs

print(f"\n🔹 APL aproximado con {sample_size} nodos de muestreo: {apl_approx:.4f}")
print(f"🔹 Pares alcanzables contabilizados: {reachable_pairs}")


|N| <built-in method vcount of Graph object at 0x703ab9967e20>    |M| <built-in method ecount of Graph object at 0x703ab9967e20>


<timed exec>:20: RuntimeWarning: Couldn't reach some vertices at src/paths/unweighted.c:368



🔹 APL aproximado con 100 nodos de muestreo: 7.3979
🔹 Pares alcanzables contabilizados: 48894654
CPU times: user 58.6 s, sys: 107 ms, total: 58.7 s
Wall time: 58.7 s
